# LangChain: Models, Prompts and Output Parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [ ]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/pedrodiamel/agents-mini-course/blob/course/books/aula_03_model_prompt.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

In [1]:
import os
import openai
from utils import format_message, show_prompt


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# Set the model variable based on the current date
llm_model = "gpt-4o-mini"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [3]:
# Ref: https://platform.openai.com/docs/api-reference/chat/create

client = openai.OpenAI()

def get_completion(prompt, model=llm_model):
    messages = [
        {
        "role": "user",
        "content": prompt
        }
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content


In [6]:
client = openai.OpenAI()

with client.chat.completions.stream(
    model=llm_model,
    messages=[{"role": "user", "content": "Explique brevemente o que é aprendizado de máquina."}],
    temperature=0,
) as stream:
    for event in stream:
        if event.type == "message.delta":
            print(event.delta, end="", flush=True)

In [4]:
response = get_completion("Explique brevemente o que é aprendizado de máquina.")
print(response)

Aprendizado de máquina é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os computadores aprendam a partir de dados. Em vez de serem programados explicitamente para realizar uma tarefa, os sistemas de aprendizado de máquina usam padrões e inferências a partir de dados para melhorar seu desempenho ao longo do tempo. Isso pode incluir tarefas como classificação, regressão, reconhecimento de padrões e tomada de decisões. O aprendizado de máquina é amplamente utilizado em diversas aplicações, como reconhecimento de voz, recomendação de produtos, detecção de fraudes e muito mais.


In [10]:
styles = ["formal and technical", "casual and friendly", "enthusiastic and persuasive", "concise and to the point", "storytelling and engaging"]
tones = ["confident", "empathetic", "urgent", "optimistic", "serious"]

In [22]:
prompt_task = """
Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.

<Task>
Redigir um email com base no contexto fornecido.
</Task>

O email precisa ter a seguinte estrutura básica:
<Structure>
1. Saudação
2. Introdução ao problema
3. Apresentação da solução
4. Benefícios da solução
5. Chamada para ação (CTA)
6. Despedida
</Structure>
"""

prompt_context = """
<Context>
Escreva um email para um cliente corporativo com o seguinte contexto:
{customer_email}
O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.
</Context>
"""

prompt_instructions = """
<Instrutions>
Use o estilo e tom especificados para redigir o email.
Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.
Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.
Não invente informações; baseie-se apenas no contexto fornecido.
Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.
</Instrutions>
"""

prompt_references = """
<Example>
Por exemplo, um email formal e técnico com tom confiante pode ser:
Asunto: Apresentação de Solução Avançada de Visão Computacional para Inspeção Industrial
Prezado(a),
Apresentamos nossa solução de visão computacional aplicada à inspeção industrial, projetada para identificar defeitos, não conformidades e desvios de processo com alta precisão e em tempo real.
A tecnologia reduz falhas humanas, aumenta a rastreabilidade e eleva a eficiência operacional, integrando-se facilmente aos sistemas existentes.
Nossa abordagem combina modelos avançados de IA, baixo tempo de resposta e escalabilidade industrial comprovada.
Colocamo-nos à disposição para agendar uma demonstração técnica e discutir a aplicação no seu ambiente produtivo.
Atenciosamente,
Costa & Silva LTDA
</Example>
"""

prompt = f"""{prompt_task}
{prompt_context}
{prompt_instructions}
{prompt_references}
"""


In [12]:
show_prompt(prompt)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                       

In [23]:
customer_email = """
Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.
"""

print(customer_email)


Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.



In [26]:
from datetime import datetime
def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %-d, %Y")

current_prompt = prompt.format(customer_email=customer_email, style=styles[2], tone=tones[1], date=get_today_str())
response = get_completion(current_prompt)

In [27]:
print(response)

**Assunto:** Apresentação de Solução Inovadora de Visão Computacional para Inspeção Industrial

Prezado(a) [Nome do Cliente],

Espero que este email o(a) encontre bem. 

Nos dias de hoje, a eficiência e a precisão na inspeção industrial são fundamentais para garantir a qualidade dos produtos e a competitividade no mercado. No entanto, muitos processos ainda dependem de inspeções manuais, que podem ser suscetíveis a erros e ineficiências. 

É com grande entusiasmo que apresentamos nossa solução de visão computacional, desenvolvida especificamente para atender às demandas da inspeção industrial. Nossa tecnologia utiliza algoritmos avançados de inteligência artificial para detectar automaticamente defeitos e não conformidades em tempo real, proporcionando uma análise precisa e confiável.

Os benefícios de nossa solução são significativos: redução de falhas humanas, aumento da rastreabilidade, otimização do tempo de produção e, consequentemente, uma melhoria na qualidade final dos produtos

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [5]:
#!pip install --upgrade langchain
#!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 4.6 MB/s eta 0:00:00


### Model

In [16]:
from langchain_openai import ChatOpenAI

In [17]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(
    temperature=0.0,
    model=llm_model
    )
chat

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x78af87b40110>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x78af69da72f0>, root_client=<openai.OpenAI object at 0x78af87b40140>, root_async_client=<openai.AsyncOpenAI object at 0x78af9082cfe0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_organization='org-5qZfsmr4Uz6f4Olo3UqM4Fgb', stream_usage=True)

In [28]:
from langchain_core.messages import HumanMessage

prompt.format(
    customer_email=customer_email,
    style=styles[2],
    tone=tones[1],
    date=get_today_str()
)

response = chat.invoke([HumanMessage(content=prompt)])
print("DONE")

DONE


In [ ]:
from utils import format_message, show_prompt

show_prompt(prompt)
format_message([response])

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                                                                                 │
│                                                       

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Assunto: Proposta de Solução para Otimização de Processos                                                       │
│                                                                                                                 │
│ Prezado [Nome do Cliente],                                                                                      │
│                                                                                                                 │
│ Espero que este email o encontre bem.                                                                           │
│                                                                                                                 │
│ Nos últimos meses, temos observado que muitas empresas enfrentam desafios significativos na otimização de seus  │
│ processos internos, o que pode resultar em ineficiências e aumento de custos. Compreendemos que a sua           │
│ organização também pode estar lidando com essas questões.                                                       │
│                                                                                                                 │
│ Para abordar esse problema, desenvolvemos uma solução inovadora que integra tecnologia de automação e análise   │
│ de dados, permitindo uma gestão mais eficiente dos recursos e uma redução significativa nos custos              │
│ operacionais. Nossa plataforma é projetada para se adaptar às necessidades específicas de cada cliente,         │
│ garantindo uma implementação suave e eficaz.                                                                    │
│                                                                                                                 │
│ Os benefícios dessa solução incluem não apenas a redução de custos, mas também a melhoria na produtividade da   │
│ equipe e a capacidade de tomar decisões mais informadas com base em dados em tempo real. Além disso, ao adotar  │
│ essa tecnologia, sua empresa se posicionará como líder em inovação no seu setor.                                │
│                                                                                                                 │
│ Gostaríamos de convidá-lo para uma demonstração personalizada, onde poderemos mostrar como nossa solução pode   │
│ ser aplicada especificamente à sua organização. Por favor, responda a este email com sua disponibilidade, e     │
│ ficaremos felizes em agendar um horário que seja conveniente para você.                                         │
│                                                                                                                 │
│ Agradeço pela atenção e estou ansioso para a oportunidade de colaborar com sua equipe.                          │
│                                                                                                                 │
│ Atenciosamente,                                                                                                 │
│                                                                                                                 │
│ [Seu Nome]                                                                                                      │
│ [Seu Cargo]                                                                                                     │
│ [Nome da Empresa]                                                                                               │
│ [Telefone]                                                                                                      │
│ [Email]                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Format output

In [30]:
from pydantic import BaseModel, Field

class EmailResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    tips: str = Field(..., description="Additional tips for improving the email.")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the email.")

class EmailImprovedResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    improved_email: str = Field(..., description="The email with the improvements applied")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the email.")

class EmailEvaluationResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    improved_email: str = Field(..., description="The email with the improvements applied")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the applied improvements.")


In [31]:
email_writer_agent = (
    chat
    .with_structured_output(EmailResponse)
)

email_with_improvements_agent = (
    chat
    .with_structured_output(EmailImprovedResponse)
)

email_evaluator_agent = (
    chat
    .with_structured_output(EmailEvaluationResponse)
)

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = email_writer_agent.invoke([HumanMessage(content=prompt)])

improvements_prompt = f"""
Voce é um assistente de IA que ajuda a aprimorar emails corporativos.

<Task>
Aprimorar um email com base nas sugestões sugeridas.
</Task>

<original_email>
{response.customer_email}
</original_email>

<suggestions>
{response.tips}
</suggestions>
"""

improvedResponse = email_with_improvements_agent.invoke([HumanMessage(content=improvements_prompt)])
evaluation_prompt = f"""
Voce é um assistente de IA que avalia melhorias em emails corporativos.

<Task>
Avaliar com nota de 1-10 as melhorias aplicadas.
</Task>

<original_email>
{improvedResponse.customer_email}
</original_email>

<improved_email>
{improvedResponse.improved_email}
</improved_email>
"""

evaluationResponse = email_evaluator_agent.invoke([HumanMessage(content=evaluation_prompt)])
print(f"O email é: {response.customer_email}")
print("\n##########\n")
print(f"As sugestões são: {response.tips}")
print("\n##########\n")
print(f"O email atualizado é: {improvedResponse.improved_email}")
print("\n##########\n")
print(f"A nota final de avaliação, é: {evaluationResponse.score}")


print("DONE")

O email é: Prezado(a) [Nome do Cliente],

Espero que este email o encontre bem. 

Identificamos que muitas empresas, como a sua, enfrentam desafios na otimização de processos e na redução de custos operacionais. Esses problemas podem impactar diretamente a eficiência e a competitividade no mercado atual.

Para ajudar a resolver essas questões, apresentamos nossa solução de automação de processos, que utiliza tecnologia de ponta para simplificar tarefas repetitivas e melhorar a produtividade da equipe. 

Com a implementação da nossa solução, você poderá observar uma redução significativa no tempo gasto em atividades manuais, além de uma diminuição nos erros operacionais. Isso não só libera sua equipe para se concentrar em tarefas mais estratégicas, mas também resulta em economia de custos a longo prazo.

Gostaríamos de convidá-lo para agendar uma demonstração personalizada, onde poderemos mostrar como nossa solução pode ser aplicada especificamente ao seu negócio e responder a quaisquer

In [ ]:
response

EmailResponse(customer_email='Assunto: Proposta de Solução para Aumento de Eficiência Operacional\n\nPrezado Sr. Almeida,\n\nEspero que este email o encontre bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a necessidade de aumentar a eficiência operacional em um ambiente de negócios cada vez mais competitivo.\n\nPara abordar essa questão, nossa equipe desenvolveu uma solução inovadora que integra tecnologia de automação com análise de dados em tempo real. Essa abordagem não apenas simplifica processos, mas também proporciona insights valiosos que podem ser utilizados para otimizar a tomada de decisões.\n\nOs benefícios dessa solução incluem a redução de custos operacionais, aumento da produtividade e a capacidade de responder rapidamente às mudanças do mercado. Com a implementação dessa tecnologia, sua empresa poderá não apenas se manter competitiva, mas também se destacar em seu setor.\n\nGostaria de convidá-lo para agendar uma demonst

In [35]:
print(prompt)
# print("Customer Email:")
# print(response.customer_email)
# print("\nTips:")
# print(response.tips)
# print("\nChain of Thought:")
# print(response.chain_of_thought)
# print("\nScore:")
# print(response.score)


Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.

<Task>
Redigir um email com base no contexto fornecido.
</Task>

O email precisa ter a seguinte estrutura básica:
<Structure>
1. Saudação
2. Introdução ao problema
3. Apresentação da solução
4. Benefícios da solução
5. Chamada para ação (CTA)
6. Despedida
</Structure>


<Context>
Escreva um email para um cliente corporativo com o seguinte contexto:
{customer_email}
O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.
</Context>


<Instrutions>
Use o estilo e tom especificados para redigir o email.
Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.
Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.
Não invente informações; baseie-se apenas no contexto fornecido.
Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.
</Instrutions>


<Example>
Por exemplo, 

### Exercisio

Para aprimorar os resultados obtidos vamos a criar um sistemas multi-agente para conseguir fazer uma análise do problema (reflexão), escrever o email a partir deste plano (escrita) e finalmente avaliar os resultados obtidos (avaliação).

- Step 1: Crie os três agentes (Agente reflexão, Agente escrita, Agente avaliação)
- Step 2: Crie um pipeline (chain) para gerar os resultados
- Step 3: Avalie e intere sobre os resultados obtidos


### Extra: Usando ollama LLMs

In [ ]:
#!pip install langchain-community

In [ ]:
from langchain_community.llms import Ollama

# Conecta ao servidor Ollama (local ou remoto)
llm = Ollama(
    model="gemma2:2b",
    base_url="http://ollama:11434"  # troque se estiver remoto
)

# Prompt simples
response = llm.invoke("Olá! Me diga um 'Olá Mundo' bem bonito em português.")

print("Resposta do modelo:")
print(response)

Resposta do modelo:
Olá, mundo! 🌎✨ 

Que a beleza e o encanto da vida fluam por todo o seu caminho. 😉  😊 

